# Support Vector Machines (SVMs)
 we will learn a linear and kernalised method of SVMs, which can be used for both regression and classification. To start with, we will focus on binary classification. We will use stochastic gradient descent (SGD) for the optimisation of the hinge loss.

We will work with the [Breast Cancer Wisconsin (Diagnostic) Data Set](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data)
<a name="section-1"></a>

## Section 1: Data Preparation


In [1]:
# necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#from google.colab import files
#upload = files.upload()
data = pd.read_csv('./data.csv')

# print shape and last 10 rows
print(data.shape)
data.tail(10)

(569, 33)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
559,925291,B,11.51,23.93,74.52,403.5,0.09261,0.10210,0.11120,0.04105,...,37.16,82.28,474.2,0.12980,0.25170,0.3630,0.09653,0.2112,0.08732,NaN
560,925292,B,14.05,27.15,91.38,600.4,0.09929,0.11260,0.04462,0.04304,...,33.17,100.20,706.7,0.12410,0.22640,0.1326,0.10480,0.2250,0.08321,NaN
561,925311,B,11.20,29.37,70.67,386.0,0.07449,0.03558,0.00000,0.00000,...,38.30,75.19,439.6,0.09267,0.05494,0.0000,0.00000,0.1566,0.05905,NaN
562,925622,M,15.22,30.62,103.40,716.9,0.10480,0.20870,0.25500,0.09429,...,42.79,128.70,915.0,0.14170,0.79170,1.1700,0.23560,0.4089,0.14090,NaN
563,926125,M,20.92,25.09,143.00,1347.0,0.10990,0.22360,0.31740,0.14740,...,29.41,179.10,1819.0,0.14070,0.41860,0.6599,0.25420,0.2929,0.09873,NaN
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.22160,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.16280,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.14180,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.26500,0.4087,0.12400,NaN
568,92751,B,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,...,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.00000,0.2871,0.07039,NaN


We can see that our data set has 569 samples and 33 columns. The column `id` can be taken as an index for our pandas dataframe and `diagnosis` is the label (either **M: malignant** or **B: benign**).

Let's prepare the data set first of all by (i) cleaning it, (ii) separating label from features, and (iii) splitting it into train and test sets.

In [3]:
# drop last column (extra column added by pd)
data_1 = data.drop(data.columns[-1], axis=1)

# set column id as dataframe index
data_2 = data_1.set_index(data['id']).drop(data_1.columns[0], axis=1)

# check
data_2.tail()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400
92751,B,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,0.1587,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039


We do a bit more preparation by converting the categorical labels into 1 for **M** and -1 for **B**.

In [4]:
# convert categorical labels to numbers
diag_map = {'M': 1.0, 'B': -1.0}
data_2['diagnosis'] = data_2['diagnosis'].map(diag_map)

# put labels and features in different dataframes
y = data_2.loc[:, 'diagnosis']
X = data_2.iloc[:, 1:]

# check
print(y.tail())
X.tail()

id
926424    1.0
926682    1.0
926954    1.0
927241    1.0
92751    -1.0
Name: diagnosis, dtype: float64


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
926424,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
926682,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
926954,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
927241,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400
92751,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,0.1587,0.05884,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039


As with any data set that has features over different ranges, it's required to standardise the data before.

In [5]:

def standardise(X):
  mu = np.mean(X, 0)
  sigma = np.std(X, 0)
  X_std = (X - mu) / sigma
  return X_std

In [6]:
X_std = standardise(X)

# check
X_std.tail()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
926424,2.110995,0.721473,2.060786,2.343856,1.041842,0.219060,1.947285,2.320965,-0.312589,-0.931027,...,1.901185,0.117700,1.752563,2.015301,0.378365,-0.273318,0.664512,1.629151,-1.360158,-0.709091
926682,1.704854,2.085134,1.615931,1.723842,0.102458,-0.017833,0.693043,1.263669,-0.217664,-1.058611,...,1.536720,2.047399,1.421940,1.494959,-0.691230,-0.394820,0.236573,0.733827,-0.531855,-0.973978
926954,0.702284,2.045574,0.672676,0.577953,-0.840484,-0.038680,0.046588,0.105777,-0.809117,-0.895587,...,0.561361,1.374854,0.579001,0.427906,-0.809587,0.350735,0.326767,0.414069,-1.104549,-0.318409
927241,1.838341,2.336457,1.982524,1.735218,1.525767,3.272144,3.296944,2.658866,2.137194,1.043695,...,1.961239,2.237926,2.303601,1.653171,1.430427,3.904848,3.197605,2.289985,1.919083,2.219635
92751,-1.808401,1.221792,-1.814389,-1.347789,-3.112085,-1.150752,-1.114873,-1.261820,-0.820070,-0.561032,...,-1.410893,0.764190,-1.432735,-1.075813,-1.859019,-1.207552,-1.305831,-1.745063,-0.048138,-0.751207


In [7]:
# split into train and test set
# stacking data X and labels y into one matrix
data_split = np.hstack((X_std, y[:, np.newaxis]))

# shuffling the rows        
np.random.shuffle(data_split)

# we split train to test as 70:30
split_rate = 0.7
train, test = np.split(data_split, [int(split_rate*(data_split.shape[0]))])

X_train = train[:,:-1]
y_train = train[:, -1]

X_test = test[:,:-1]
y_test = test[:, -1]

y_train = y_train.astype(float)
y_test = y_test.astype(float)

# insert 1 in every row for intercept b
X_train_intercept = np.hstack((X_train, np.ones((len(X_train),1)) ))
X_test_intercept = np.hstack((X_test, np.ones((len(X_test),1)) ))    

C:\Users\44746\AppData\Local\Temp\ipykernel_29712\73579034.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  data_split = np.hstack((X_std, y[:, np.newaxis]))


<a name="section-2"></a>

## Section 2: Linear SVM Formulation

We start with defining the hinge loss as
$$
\mathcal L (\boldsymbol w) = \frac{1}{2} \| \boldsymbol w \|^2 + \lambda \sum_{i=1}^n \max \bigg( 0, 1-y_i (x^{(i)} \cdot \boldsymbol w  + b) \bigg) \, .
$$
where $\boldsymbol w$ is the vector of weights, $\lambda$ the regularisation parameter, and $b$ the intercept which is included in our `X` as an additional column of $1$'s.

In [8]:
def compute_cost(w, X, y, regul_strength=1e5):
  n = X.shape[0]
  distances = 1 - y * (X @ w)
  distances[distances < 0] = 0  # equivalent to max(0, distance)
  hinge = regul_strength * distances.mean()
    
  # calculate cost
  return 0.5 * np.dot(w, w) + hinge -0.5*w[-1]**2

<a name="section-3"></a>

## Section 3: SVM Optimization using SGD

One way to optimize the cost is by using stochastic gradient descent (SGD) algorithm. In order to use SGD, we need to implement a function for the cost gradients with respect to $\boldsymbol w$.

In [9]:
# calculate gradient of cost
def calculate_cost_gradient(w, X_batch, y_batch, regul_strength=1e6):
  # if only one example is passed
  if type(y_batch) == np.float64:
      y_batch = np.asarray([y_batch])
      X_batch = np.asarray([X_batch])  # gives multidimensional array

  distance = 1 - (y_batch * (X_batch @ w))
  dw = np.zeros(len(w))
  
  we = w.copy() # So as not to overwrite w
  we[-1] = 0 # So as not to have b in its derivative when adding the weights in di
    
  for ind, d in enumerate(distance):
      if max(0, d)==0:
          di = we # derivative of first term
      else:
          di = we - (regul_strength * y_batch[ind] * X_batch[ind])
      dw += di

  return dw/len(y_batch)  # average

Both of the two previous functions are then used in SGD to update the weights iteratively with a given learning rate $\alpha$. We also implement a stop criterion that ends the learning as soon as the cost function has not changed more than a manually determined percentage.

We know that the learning happens through updating the weights according to
$$
\boldsymbol w = \boldsymbol w - \alpha \frac{\partial \mathcal L}{\partial \boldsymbol w}
$$

where $\frac{\partial \mathcal L}{\partial \boldsymbol w}$ is the gradient of the hinge loss we have computed in the previous cell.

In [10]:
def sgd(X, y, max_iterations=2000, stop_criterion=0.01, learning_rate=1e-5, regul_strength=1e6, print_outcome=False):
  
  # initialise zero weights
  weights = np.zeros(X.shape[1])
  nth = 0
  # initialise starting cost as infinity
  prev_cost = np.inf
  
  # stochastic gradient descent
  indices = np.arange(len(y))

  for iteration in range(1, max_iterations):
    # shuffle to prevent repeating update cycles
    np.random.shuffle(indices)
    X, y = X[indices], y[indices]
    
    for xi, yi in zip(X, y):
      descent = calculate_cost_gradient(weights, xi, yi, regul_strength)
      weights = weights - (learning_rate * descent)

    # convergence check on 2^n'th iteration
    if iteration==2**nth or iteration==max_iterations-1:
      # compute cost
      cost = compute_cost(weights, X, y, regul_strength) 
      if print_outcome:
        print("Iteration is: {}, Cost is: {}".format(iteration, cost))
      # stop criterion
      if abs(prev_cost - cost) < stop_criterion * prev_cost:
        return weights
        
      prev_cost = cost
      nth += 1
  
  return weights

Now, we can take these functions and train a linear SVM with our training data.

In [11]:
# train the model
lam=10
w = sgd(X_train_intercept, y_train, max_iterations=2000, stop_criterion=0.001, learning_rate=1e-5, regul_strength=lam, print_outcome=True)
print("Training finished.")

Iteration is: 1, Cost is: 6.9907420010698305
Iteration is: 2, Cost is: 4.854995029501536
Iteration is: 4, Cost is: 3.1103565432405427
Iteration is: 8, Cost is: 2.1821378738054347
Iteration is: 16, Cost is: 1.7133550780898186
Iteration is: 32, Cost is: 1.4424555736338986
Iteration is: 64, Cost is: 1.3072345222727204
Iteration is: 128, Cost is: 1.2468862612954499
Iteration is: 256, Cost is: 1.216749107406207
Iteration is: 512, Cost is: 1.2123703308353666
Iteration is: 1024, Cost is: 1.2116123712948907
Training finished.


To evaluate the mean accuracy in both train and test set, we write a small function called `score`.

In [12]:
def score(w, X, y):
  y_preds = np.sign(X @ w)
  return np.mean(y_preds == y)

print("Accuracy on training set: {}".format(score(w, X_train_intercept, y_train)))
print("Accuracy on test set: {}".format(score(w, X_test_intercept, y_test)))

Accuracy on training set: 0.9824120603015075
Accuracy on test set: 0.9766081871345029


<a name="section-4"></a>

## Section 4: Model Evaluation via *T*-fold Cross Validation

Now we repeat the same procedure as above but do not only have one train-test split, but multiple in a *T*-fold cross validation method.

In [13]:
def cross_val_split(N, num_folds):
  fold_size = N // num_folds
  index_perm = np.random.permutation(np.arange(N))
  folds = []
  for k in range(num_folds):
    folds.append(index_perm[k*fold_size:(k+1)*fold_size])
  return folds

In [14]:
# evaluate
folds = cross_val_split(train.shape[0], 5)
folds

[array([266, 249,  93,  45, 384, 172,  23, 229, 347,  46, 252, 270, 175,
        207, 343,  30,  83,  37, 123, 344, 239, 387,  12, 153, 165,  21,
        298, 285,  56, 178, 381, 138, 281, 221,  19, 364, 149,  42, 341,
        280,  28, 127, 278, 105, 299, 109,  57, 353, 392, 335, 118, 304,
        176,   8,  67, 264, 360, 136, 276, 312, 132, 187, 289, 262,  75,
         87,   5, 224, 204, 368,  35, 255,  98, 338, 263, 254, 300, 163,
        107]),
 array([226, 339, 277,  99, 356, 245, 160,  50, 143, 342, 358,  17, 332,
        305, 275, 108, 355, 345,  65, 352, 151, 391, 328, 192, 382, 334,
         72, 348,  38,  32, 265, 273, 309, 120,  39, 286, 134,  97,  88,
        104,   1, 379, 366,  85,  64,  29, 386, 101,  53, 231, 248, 283,
         51,  91, 156, 112,  81, 351,  74,  62, 125,  71, 121, 306, 359,
         54, 198, 147, 378, 219, 268, 395, 182,   2,  95, 257, 131,  33,
        148]),
 array([324, 201, 144, 329, 116,  58, 110, 233, 367, 314, 191, 111, 159,
        146, 318, 369

In [15]:
def cross_val_evaluate(data, num_folds):
  folds = cross_val_split(data.shape[0], num_folds)

  train_scores = []
  val_scores = []
  
  for i in range(len(folds)):
    print('Fold', i+1)

    val_indices = folds[i]
    # define the training set
    train_indices = list(set(range(data.shape[0])) - set(val_indices))

    X_train = data[train_indices,  :-1]  
    y_train = data[train_indices, -1]
    
    # define the validation set
    X_val = data[val_indices,  :-1]  
    y_val = data[val_indices, -1]  
    
    # insert 1 in every row for intercept b
    X_train = np.hstack((X_train, np.ones((len(X_train),1)) ))
    X_val = np.hstack((X_val, np.ones((len(X_val),1)) ))  

    # train the model
    w = sgd(X_train, y_train, max_iterations=1025, stop_criterion=0.01, learning_rate=1e-5, regul_strength=1e3)
    print("Training finished.")

    # evaluate
    train_score = score(w, X_train, y_train)
    val_score = score(w, X_val, y_val)
    print("Accuracy on training set #{}: {}".format(i+1, train_score))
    print("Accuracy on validation set #{}: {}".format(i+1, val_score))

    train_scores.append(train_score)
    val_scores.append(val_score)

  return train_scores, val_scores

In [16]:
train_scores, val_scores = cross_val_evaluate(train, 5)

Fold 1
Training finished.
Accuracy on training set #1: 0.987460815047022
Accuracy on validation set #1: 0.9873417721518988
Fold 2
Training finished.
Accuracy on training set #2: 0.987460815047022
Accuracy on validation set #2: 1.0
Fold 3
Training finished.
Accuracy on training set #3: 0.9937304075235109
Accuracy on validation set #3: 0.9367088607594937
Fold 4
Training finished.
Accuracy on training set #4: 0.9968652037617555
Accuracy on validation set #4: 0.9620253164556962
Fold 5
Training finished.
Accuracy on training set #5: 0.9905956112852664
Accuracy on validation set #5: 0.9620253164556962


we compute the mean accuracy.

In [17]:
print(np.mean(train_scores), np.mean(val_scores))

0.9912225705329154 0.969620253164557


### Subsection 4.1: Kernelised SVM

In the follwing, we implement a soft-margin kernelised SVM classifier with a non-linear kernel. Here, we use a Gaussian kernel:
$$k(x,y|\sigma) = e^{-\frac{||x-y||^2}{\sigma}}$$

In [18]:
# First we need a function to calculate the kernel given the data #
def kernel_matrix(X1,X2,sigma):

    n1,m1 = X1.shape
    n2,m2 = X2.shape
    kernel = np.zeros((n1,n2))

    # Here we define a Gaussian Radial Basis Function Kernel #
    for i in range(n1):
        exponent = np.linalg.norm(X2 - X1[i],axis=1)**2
        kernel[i,:] = np.exp(-exponent/sigma) 
        
    return kernel

Having defined the kernel, we use this to compute the cost below.

In [19]:
def compute_cost_kernel(u, K, y, regul_strength=1e3,intercept=0):

    # Here I define the hinge cost with the kernel trick. NB: the intercept should be kept separate #
    
    distances = 1 - (y)*(K@u + intercept) 
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge = regul_strength * distances.mean()

    # calculate cost
    return 0.5 * np.dot(u,K@u) + hinge

As we have seen in the lecture notes, the kernel trick can be implemented in the primal-form problem by defining a new loss function:

$$L(\mathbf{u},b) = \frac{1}{2}\mathbf{u}^{\rm{T}}\mathbf{K} \mathbf{u} + \lambda \sum_{i=1}^N  \max \Big\{0, 1-y^{(i)}(\mathbf{K}^{(i)}\mathbf{u} + b)\Big\}$$

where $\mathbf{K}$ is the matrix containing the kernel functions, i.e.
 $\mathbf{K}_{ij} = k(\mathbf{x}^{(i)},\mathbf{x}^{(i)})$. To perform the optimisation, we simply modify the functions introduced above. Note that we will use $X_{\mathrm{train}}$ and $X_{\mathrm{test}}$ without the additional vector of ones. We had previously included this vector of ones to learn the intercept term $b$, but within the new formulation, one cannot readily employ this trick. For simplicity, we will drop the intercept by setting $b=0$ here. We are hinting at how you could implement the intercept, but we have deliberately left out a few lines of code for you to work on in the course work (2023).
 
Following the steps above, we want to optimize the cost is by using SGD. First, we thus create a function that computes the gradients.

In [20]:
def calculate_cost_gradient_kernel(u, K_batch, y_batch, regul_strength=1e3,intercept=0):

    # if only one example is passed
    if type(y_batch) == np.float64 or type(y_batch) == np.int32:
        y_batch = np.asarray([y_batch])
        K_batch = np.asarray([K_batch])  # gives multidimensional array
    
    distance = 1 - (y_batch * (K_batch @ u + intercept)) 
    dw = np.zeros(len(u))

    # define the gradient with the hinge loss #
    for ind, d in enumerate(distance):
        if max(0, d)==0:
            di = K_batch@u 
        else:
            di = K_batch@u - (regul_strength * y_batch[ind] * K_batch[ind]) 
        dw += di

    return dw/len(y_batch)

We can now use the functions above in SGD.

In [21]:
def sgd_kernel(K, y, batch_size=32, max_iterations=4000, stop_criterion=0.001, learning_rate=1e-4, regul_strength=1e3, print_outcome=False):

    # initialise zero u and intercept
    u = np.zeros(K.shape[1])
    intercept=0
    
    nth = 0
    # initialise starting cost as infinity
    prev_cost = np.inf
    
    # stochastic gradient descent
    indices = np.arange(len(y))
    for iteration in range(1, max_iterations):
        # shuffle to prevent repeating update cycles
        np.random.shuffle(indices)
        batch_idx = indices[:batch_size]
        K_b, y_b = K[batch_idx], y[batch_idx]
        for ki, yi in zip(K_b, y_b):
            ascent = calculate_cost_gradient_kernel(u, ki, yi, regul_strength, intercept) 
            u = u - (learning_rate * ascent)
        
        # convergence check on 2^n'th iteration
        if iteration==2**nth or iteration==max_iterations-1:
            # compute cost
            cost = compute_cost_kernel(u, K, y, regul_strength, intercept)
            if print_outcome:
                print("Iteration is: {}, Cost is: {}".format(iteration, cost))
            # stop criterion
            if abs(prev_cost - cost) < stop_criterion * prev_cost:
                return u, intercept
            
            prev_cost = cost
            nth += 1
    
    return u, intercept

Finally, let's fix the regularisation.

In [22]:
reg=1
for sigma in [1,2,5,10]:
    
    print('For sigma = ' + str(sigma))
    K_train = kernel_matrix(X_train,X_train, sigma)

    u,b = sgd_kernel(K_train, y_train, batch_size=128, max_iterations=2000, stop_criterion=0.001, learning_rate=1e-5, regul_strength=1e3, print_outcome=False)

    def score(u, X, y, sigma, intercept):
        ## now I define the kernel containing test and train data ##
        K_test = kernel_matrix(X,X_train, sigma)
        
        ## The 
        y_preds = np.sign(K_test@u + intercept)
        
        return np.mean(y_preds == y) 

    print("Accuracy on training set: {}".format(score(u, X_train, y_train, sigma, b)))
    print("Accuracy on test set: {}".format(score(u, X_test, y_test, sigma, b)))

For sigma = 1
Accuracy on training set: 1.0
Accuracy on test set: 0.9649122807017544
For sigma = 2
Accuracy on training set: 1.0
Accuracy on test set: 0.9707602339181286
For sigma = 5
Accuracy on training set: 0.9974874371859297
Accuracy on test set: 0.9766081871345029
For sigma = 10
Accuracy on training set: 0.9974874371859297
Accuracy on test set: 0.9766081871345029
